In [1]:
import pandas as pd
import requests as re
from config import fmp
# import plotly.graph_objects as go

In [4]:
api_key = fmp['api_key']
ticker = 'NVDA'

resp_is = re.get(f'https://financialmodelingprep.com/api/v3/income-statement/{ticker}?limit=120&apikey={api_key}').json()
resp_is = pd.DataFrame(resp_is, columns=['date', 'revenue', 'netIncome', 'operatingIncome'])

resp_bs = re.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?limit=120&apikey={api_key}').json()
resp_bs = pd.DataFrame(resp_bs, columns=['date', 'totalStockholdersEquity', 'totalLiabilities', 'longTermDebt'])

resp_cf = re.get(f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?limit=120&apikey={api_key}').json()
resp_cf = pd.DataFrame(resp_cf, columns=['date', 'dividendsPaid'])
resp_cf['dividendsPaid'] = -resp_cf['dividendsPaid']

df = resp_is.merge(resp_bs, on='date').merge(resp_cf, on='date')
df['date'] = pd.to_datetime(df['date'])
# df.set_index('date', inplace=True)

df.head()

,date,revenue,netIncome,operatingIncome,totalStockholdersEquity,totalLiabilities,longTermDebt,dividendsPaid
0,2020-01-26,10918000000,2796000000,2846000000,12204000000,5111000000,1991000000,390000000
1,2019-01-27,11716000000,4141000000,3804000000,9342000000,3950000000,1988000000,371000000
2,2018-01-28,9714000000,3047000000,3210000000,7471000000,3770000000,1985000000,341000000
3,2017-01-29,6910000000,1666000000,1934000000,5762000000,4079000000,2020000000,261000000
4,2016-01-31,5010000000,614000000,878000000,4469000000,2901000000,0,213000000


In [70]:
df['Equity+Dividends'] = df['totalStockholdersEquity'] + df['dividendsPaid']

In [71]:
df['roe'] = df['netIncome'] / df['totalStockholdersEquity']
df['roic'] = df['netIncome'] / (df['totalStockholdersEquity'] + df['longTermDebt'])

In [72]:
cols = ['revenue', 'netIncome', 'operatingIncome', 'Equity+Dividends']
cols_yoy = [col + '_yoy' for col in cols]

df[cols_yoy] = (df[cols] - df[cols].shift(-1)) / abs(df[cols].shift(-1))
df

,date,revenue,netIncome,operatingIncome,totalStockholdersEquity,totalLiabilities,longTermDebt,dividendsPaid,Equity+Dividends,roe,roic,revenue_yoy,netIncome_yoy,operatingIncome_yoy,Equity+Dividends_yoy
0,2020-01-26,10918000000,2796000000,2846000000,12204000000,5111000000,1.991000e+09,390000000,12594000000,0.229105,0.196971,-0.068112,-0.324801,-0.251840,0.296613
1,2019-01-27,11716000000,4141000000,3804000000,9342000000,3950000000,1.988000e+09,371000000,9713000000,0.443267,0.365490,0.206094,0.359042,0.185047,0.243344
2,2018-01-28,9714000000,3047000000,3210000000,7471000000,3770000000,1.985000e+09,341000000,7812000000,0.407844,0.322229,0.405789,0.828932,0.659772,0.297028
3,2017-01-29,6910000000,1666000000,1934000000,5762000000,4079000000,2.020000e+09,261000000,6023000000,0.289136,0.214084,0.379242,1.713355,1.202733,0.286416
4,2016-01-31,5010000000,614000000,878000000,4469000000,2901000000,0.000000e+00,213000000,4682000000,0.137391,0.137391,NaN,NaN,NaN,NaN


In [88]:
# To do - calculate avg growth rates and plot resultsed
metrics = ['revenue_yoy', 'netIncome_yoy', 'operatingIncome_yoy', 'Equity+Dividends_yoy']
growth_rates = pd.DataFrame(metrics, columns = ['Metric'])

ranges = [2, 5] #slicing is inclusive with df.loc
for r in ranges:
    growth_rates[f'{r} yr avg'] = df.loc[:r, metrics].mean().values

growth_rates.head()

,Metric,2 yr avg,5 yr avg
0,revenue_yoy,0.181257,0.230753
1,netIncome_yoy,0.287724,0.644132
2,operatingIncome_yoy,0.197660,0.448928
3,Equity+Dividends_yoy,0.278995,0.280850
